In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta
from vnstock3 import Vnstock

# Setting

In [2]:
rf = {'2019': 0.0451, '2020': 0.0286, '2021':0.023,'2022': 0.0335,'2023':0.0321}
rf = pd.Series(rf)
rf_cal = rf.mean()
rf_2022 = 0.0335

In [3]:
pd.set_option('display.max_columns', None) 

In [4]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
OBV_PERIOD = 5
initial_investment = 100000000

In [5]:
win_rate = 0.525862069
loss_rate = 1 - win_rate
mean_profit = 0.1539675969
mean_loss = 0.2086126
stop_loss = 0.08

In [6]:
def kelly_criterion(p, q, profit, loss):
    b = (profit * 100_000_000)/(loss * 100_000_000)
    f = (b*p - q)/b
    return f

In [7]:
f =kelly_criterion(win_rate,loss_rate,mean_profit,mean_loss)
f

-0.11655327381250193

In [8]:
high_ESG_group = ['CTD', 'DHG', 'DPM', 'FPT', 'GAS', 'MBB']
low_ESG_group = ['NVL', 'PNJ', 'REE', 'SBT', 'SSI', 'STB', 'VIC', 'VNM']
non_ESG_group = ['CII', 'CTG', 'EIB', 'GMD', 'HDB', 'HPG', 'MSN', 'MWG', 'ROS', 'SAB', 'TCB', 'VCB', 'VHM', 'VJC', 'VPB', 'VRE']


In [9]:
companies = ['VCB']

# Calculate indicators

In [10]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
    df['OBV_Slope'] = df['OBV'].diff(periods=OBV_PERIOD)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

In [11]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI across 30 and OBV rise
    df.loc[(df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0), 'Signal'] = 1

    #Sell Signals: RSI across 70 and OBV down 
    df.loc[(df['Previous_RSI'] > RSI_OVERBOUGHT) & (df['RSI'] <= RSI_OVERBOUGHT) & (df['OBV_Slope'] < 0), 'Signal'] = -1

    return df

In [12]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [13]:
def calculate_daily_risk_free_rate(annual_rate_decimal, trading_days=252):
    # Convert percentage to decimal
    # Calculate daily risk-free rate using the compound formula
    daily_rate = (1 + annual_rate_decimal) ** (1 / trading_days) - 1
    
    # Alternatively, for a simple calculation:
    # daily_rate = annual_rate_decimal / trading_days
    
    return daily_rate

# Example usage
annual_yield = 0.0326  # Example annual yield of 3%
daily_risk_free_rate = calculate_daily_risk_free_rate(rf_cal)
daily_risk_free_rate_2022 = calculate_daily_risk_free_rate(rf_2022)


# Backtest

In [14]:
def simulate_investment(
    ticker, win_rate, loss_rate, mean_profit, mean_loss, 
    sell_fraction, start_date, end_date, f_star=1
):
    try:
        # Initialize trade counters and portfolio metrics
        number_of_buying_trades = 0
        number_of_selling_trades = 0
        cash = initial_investment
        holdings = 0
        portfolio_values = []

        # Load stock data and calculate indicators
        data = vn.stock_historical_data(ticker, start_date, end_date, resolution='1D', type='stock', source='TCBS')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = macd_strategy(data)
        data = data.dropna(subset=['time'])
        data = data.drop_duplicates(subset=['time', 'open', 'high', 'low', 'close', 'volume'], keep='first')
        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i, current_date in enumerate(data.index):
            current_price = data['close'].iloc[i]

            # Handle pending T+2 settlements
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            # Avoid trades in January 2024
            if current_date.month == 1 and current_date.year == 2024:
                portfolio_values.append(cash + holdings * current_price)
                continue

            # Buy if there's a buy signal and cash allows
            if current_date in buy_signals:
                allocation = cash * f_star
                shares_to_buy = int(allocation // current_price)
                total_cost = shares_to_buy * current_price
                if shares_to_buy > 0 and cash >= total_cost:
                    cash -= total_cost
                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    pending_buy_shares[settlement_date] = pending_buy_shares.get(settlement_date, 0) + shares_to_buy
                    last_buy_price = current_price
                    number_of_buying_trades += 1

            # Sell if there's a sell signal and holdings allow
            if holdings > 0 and current_date in sell_signals:
                shares_to_sell = int(holdings * sell_fraction)
                revenue = shares_to_sell * current_price
                holdings -= shares_to_sell
                settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                pending_sell_revenue[settlement_date] = pending_sell_revenue.get(settlement_date, 0) + revenue
                number_of_selling_trades += 1

            # Update portfolio value
            portfolio_values.append(cash + holdings * current_price)

        # Finalize portfolio values including pending settlements
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)
            portfolio_values.append(cash + holdings * data['close'].iloc[-1])
            final_date += BDay(1)

        # Adjust portfolio values to match data index length
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))
    
        # Add portfolio values to data frame
        data['Portfolio_Value'] = portfolio_values
        data['value'] = portfolio_values
        data['Number_of_Buying_Trades'] = number_of_buying_trades
        data['Number_of_Selling_Trades'] = number_of_selling_trades

        # Set `value` to NaN if `Signal` is non-zero, for both current and next row
        for i in range(len(data) - 1):  # Avoid index out of bounds for i+1
            if data['Signal'].iloc[i] != 0:  # Use iloc to access a single value
                data.loc[data.index[i], 'value'] = np.nan
                data.loc[data.index[i + 1], 'value'] = np.nan
                

        # Forward fill `value` where needed
        data['value'] = data['value'].ffill()

        # Calculate returns and performance metrics
        data['Daily_Return'] = data['value'].pct_change()
        data['Accumulated_Profit'] = data['value'] - initial_investment
        data['Running_Max'] = data['value'].cummax()  # Track the running max portfolio value
        data['Drawdown'] = (data['value'] - data['Running_Max']) / data['Running_Max']  # Calculate drawdown
        
        return data

    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()


In [15]:
def calculate_sharpe_ratio(data, risk_free_rate=0.01):
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Daily_Return'].dropna()
    # Calculate average return and standard deviation of returns
    average_return = daily_returns.mean()
    mean_return = average_return *252
    std_deviation = daily_returns.std()
    volatility = std_deviation * np.sqrt(252)
    # Calculate the Sharpe Ratio
    sharpe_ratio = (mean_return - risk_free_rate) / volatility if volatility != 0 else 0

    return sharpe_ratio

In [16]:
def calculate_sortino_ratio(data,rf=0.01):  # Target return can be set to risk-free rate
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Daily_Return'].dropna()

    # Calculate average return
    average_return = daily_returns.mean()
    mean_return = average_return *252
    # Calculate downside returns (returns below the target return)
    downside_returns = daily_returns[daily_returns < 0]

    # Calculate downside deviation
    downside_deviation = downside_returns.std()*np.sqrt(252)if not downside_returns.empty else np.nan

    # Calculate the Sortino Ratio
    sortino_ratio = (mean_return - rf) / downside_deviation if downside_deviation > 0 else np.nan

    return sortino_ratio

thêm phần f star để điều chỉnh kelly ở code phía dưới

In [17]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit,\
                                 mean_loss, sell_fraction, start_date, end_date, rf):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, \
                                     mean_loss, sell_fraction, start_date=start_date, end_date=end_date, \
                                        f_star=f)
        if not result.empty:
            # Calculate the Sharpe Ratio for the result
            sharpe_ratio = calculate_sharpe_ratio(result,risk_free_rate=rf)
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / initial_investment * 100,
                'Number of Buying Trades': result['Number_of_Buying_Trades'].iloc[-1],
                'Number of Selling Trades': result['Number_of_Selling_Trades'].iloc[-1],
                'Sharpe Ratio': sharpe_ratio,
                'Sortino Ratio': calculate_sortino_ratio(result,rf=rf),
                'MDD': result['Drawdown'].min(),
            })
    return pd.DataFrame(results)

# Kelly 2019-2024

In [18]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss,\
                                        sell_fraction, start_date='2018-12-28' ,end_date='2024-01-05', rf=rf_cal)
high_esg = pd.DataFrame(results_df)
print(high_esg)
average_rate_of_return = results_df['Rate of Return'].mean()
average_profit = results_df[results_df['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = results_df[results_df['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}','avg sharpe:',results_df['Sharpe Ratio'].mean(),'avg sortino:',results_df['Sortino Ratio'].mean())

Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD              100000000           0.0             0.0   
1     DHG              100000000           0.0             0.0   
2     DPM              100000000           0.0             0.0   
3     FPT              100000000           0.0             0.0   
4     GAS              100000000           0.0             0.0   
5     MBB              100000000           0.0             0.0   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        0                         0             0   
1                        0                         0             0   
2              

In [19]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss,\
                                             sell_fraction, start_date='2018-12-29', end_date='2024-01-05', \
                                                rf=rf_cal)
low_esg_kelly = pd.DataFrame(low_esg_kelly)
print(low_esg_kelly)
average_rate_of_return = low_esg_kelly['Rate of Return'].mean()
average_profit = low_esg_kelly[low_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly[low_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}


In [20]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2018-12-29', end_date='2024-01-05', rf=rf_cal)
non_esg_kelly = pd.DataFrame(non_esg_kelly)
print(non_esg_kelly)
average_rate_of_return = non_esg_kelly['Rate of Return'].mean()
average_profit = non_esg_kelly[non_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly[non_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Time range is 1833 days. Looping through 6 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
E

In [21]:
high_esg['ESG'] ='High'
low_esg_kelly['ESG'] = 'Low'
non_esg_kelly['ESG'] = 'Non'

In [22]:
merged = pd.concat([high_esg, low_esg_kelly, non_esg_kelly]) 
merged['period'] = '2019-2024' 

# Kelly 2022

In [23]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
high_esg_2022 = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=rf_2022)
high_esg_2022 = pd.DataFrame(high_esg_2022)
print(high_esg_2022)
average_rate_of_return = high_esg_2022['Rate of Return'].mean()
average_profit = high_esg_2022[high_esg_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = high_esg_2022[high_esg_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
high_esg_2022['ESG'] ='High'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD              100000000           0.0             0.0   
1     DHG              100000000           0.0             0.0   
2     DPM              100000000           0.0             0.0   
3     FPT              100000000           0.0             0.0   
4     GAS              100000000           0.0             0.0   
5     MBB              100000000           0.0             0.0   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        0                         0             0   
1                        0                         0             0   
2                    

In [24]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly_2022 = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=rf_2022)
low_esg_kelly_2022 = pd.DataFrame(low_esg_kelly_2022)
print(low_esg_kelly_2022)
average_rate_of_return = low_esg_kelly_2022['Rate of Return'].mean()
average_profit = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
low_esg_kelly_2022['ESG'] = 'Low'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL              100000000           0.0             0.0   
1     PNJ              100000000           0.0             0.0   
2     REE              100000000           0.0             0.0   
3     SBT              100000000           0.0             0.0   
4     SSI              100000000           0.0             0.0   
5     STB              100000000           0.0             0.0   
6     VIC              100000000           0.0             0.0   
7     VNM              100000000           0.0             0.0  

In [25]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly_2022 = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=rf_2022)
non_esg_kelly_2022 = pd.DataFrame(non_esg_kelly_2022)
print(non_esg_kelly_2022)
average_rate_of_return = non_esg_kelly_2022['Rate of Return'].mean()
average_profit = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
non_esg_kelly_2022['ESG'] = 'Non'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
   Company  Final Portfolio Value  Total Profit  Rate of Return  \
0      CII              100000000           0.0             0.0   
1      CTG              100000000           0.0   

In [26]:
merged_2022 = pd.concat([high_esg_2022, low_esg_kelly_2022, non_esg_kelly_2022])
merged_2022['period'] = '2022-2023'
merged_2022

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,100000000,0.0,0.0,0,0,0,NaN,0.0,High,2022-2023
1,DHG,100000000,0.0,0.0,0,0,0,NaN,0.0,High,2022-2023
2,DPM,100000000,0.0,0.0,0,0,0,NaN,0.0,High,2022-2023
3,FPT,100000000,0.0,0.0,0,0,0,NaN,0.0,High,2022-2023
4,GAS,100000000,0.0,0.0,0,0,0,NaN,0.0,High,2022-2023
5,MBB,100000000,0.0,0.0,0,0,0,NaN,0.0,High,2022-2023
0,NVL,100000000,0.0,0.0,0,0,0,NaN,0.0,Low,2022-2023
1,PNJ,100000000,0.0,0.0,0,0,0,NaN,0.0,Low,2022-2023
2,REE,100000000,0.0,0.0,0,0,0,NaN,0.0,Low,2022-2023
3,SBT,100000000,0.0,0.0,0,0,0,NaN,0.0,Low,2022-2023


# Gộp 2 cái

In [27]:
merged_all = pd.concat([merged, merged_2022])
merged_all= merged_all.reset_index()
merged_all['total_trades']= merged_all['Number of Buying Trades'] + merged_all['Number of Selling Trades']
merged_all.to_csv('obv kelly.csv', index=False)

In [28]:
kelly = pd.read_csv('obv kelly.csv')
no_kelly = pd.read_csv('obv no kelly.csv')

In [29]:
kelly['Kelly'] = 'Yes'
no_kelly['Kelly'] = 'No'
double_merged = pd.concat([kelly, no_kelly])
double_merged.to_excel('obv.xlsx', index=False)